In [3]:
import pandas as pd
import numpy as np
import glob, os
import time
import pickle
STATIC_FIELDS = ['RecordID', 'Age', 'Gender', 'Height', 'ICUType']
FIELDS = ['RecordID', 'TimeStamp', 'Albumin', 'ALP', 'ALT', 'AST', 'Bilirubin', 'BUN', 'Cholesterol',
          'Creatinine', 'DiasABP', 'FiO2', 'GCS', 'Glucose', 'HCO3', 'HCT',
          'HR', 'K', 'Lactate', 'MAP', 'MechVent', 'Mg', 'Na', 'NIDiasABP',
          'NIMAP', 'NISysABP', 'PaCO2', 'PaO2', 'pH', 'Platelets', 'RespRate',
          'SaO2', 'SysABP', 'Temp', 'TroponinI', 'TroponinT', 'Urine', 'WBC',
          'Weight']
df_all = pd.DataFrame(columns=FIELDS)
df_static_all = pd.DataFrame(columns=STATIC_FIELDS)

In [4]:
file_count = 0
unq_ids = []
time_i = time.time()
for filename in glob.glob('set-a/*.txt'):
#     if j % 100 == 0:
#         print j, str(time.time() - time_i)
    file_count += 1
    file = open(filename)
    file_list = [] 
    for line in file:
        file_list.append(line.strip("\n").split(","))
    file_list_df = pd.DataFrame(file_list, columns=['Time', 'Parameter', 'Value'])
    unq_times = list(set(list(file_list_df['Time'])) - set(['Time']))
    unq_times = sorted(unq_times)
    id = file_list_df.loc[file_list_df.Parameter=='RecordID', ['Value']].values[0][0]
    unq_ids.append(id)
    a = np.empty((len(unq_times),len(FIELDS),))
    a[:] = np.NAN
    a[:,0] = id
    df = pd.DataFrame(a,columns=FIELDS)
    df['TimeStamp'] = unq_times
    for i in range(len(file_list)):
        try:
            timestamp = file_list[i][0]
            var = file_list[i][1]
            val = file_list[i][2]
            df.loc[(df.TimeStamp == timestamp), [var]] = val
        except KeyError:
            continue
    age = file_list_df.loc[file_list_df.Parameter=='Age', ['Value']].values[0][0]
    Gender = file_list_df.loc[file_list_df.Parameter=='Gender', ['Value']].values[0][0]
    Height = file_list_df.loc[file_list_df.Parameter=='Height', ['Value']].values[0][0]
    ICUType = file_list_df.loc[file_list_df.Parameter=='ICUType', ['Value']].values[0][0]    
    df_static = pd.DataFrame(columns=STATIC_FIELDS)
    data = pd.DataFrame([[id, age, Gender, Height, ICUType]], columns=STATIC_FIELDS)
    df_static =  df_static.append(data)
    df_static_all = df_static_all.append(df_static)
    df_all = df_all.append(df)

In [48]:
unq_ids[0]

'134389'

In [5]:
a = [df_all,  df_static_all]
with open('filename.pickle', 'wb') as handle:
    pickle.dump(a, handle, protocol = pickle.HIGHEST_PROTOCOL)

In [6]:
with open('filename.pickle', 'rb') as handle:
    [df_all,  df_static_all] = pickle.load(handle)

In [7]:
df_all['TimeStamp'] = df_all['TimeStamp'] + ':00'
df_all['TimeStamp'] = pd.to_timedelta(df_all['TimeStamp'])
df_all = df_all.reset_index(drop=True)

In [8]:
time_i = time.time()
i = 0
for id_ in unq_ids:
    if i%100 == 0:
        print i, time_i - time.time()
    i = i+1
    id1 = np.empty((1,39))
    id1 = id1.astype(str)
    id1[:] = np.NAN
    id1[0][0] = id_
    id1[0][1] = '48:59:59'
    df_add = pd.DataFrame(id1, columns=df_all.columns)
    df_all = df_all.append(df_add)

0 -0.000513076782227
100 -20.1738269329
200 -36.713506937
300 -53.2364230156
400 -69.7846729755
500 -86.2604291439
600 -102.707442999
700 -119.119737148
800 -135.550946951
900 -151.980947018
1000 -168.435681105
1100 -184.917546034
1200 -201.476398945
1300 -218.089975119
1400 -234.714628935
1500 -251.404015064
1600 -268.068283081
1700 -284.731822014
1800 -301.44558692
1900 -318.22010994
2000 -335.120018959
2100 -351.907516956
2200 -368.737652063
2300 -385.542169094
2400 -402.396080017
2500 -419.823616028
2600 -436.815293074
2700 -453.793735027
2800 -471.57111311
2900 -488.769581079
3000 -505.94066596
3100 -523.33919692
3200 -541.45450592
3300 -559.127493143
3400 -577.370937109
3500 -594.520870924
3600 -4783.97577715
3700 -4804.92363095
3800 -4824.39377999
3900 -4844.04007411


In [9]:
df_all['TimeStamp'] = pd.to_timedelta(df_all['TimeStamp'])
df_all = df_all.reset_index(drop=True)

In [10]:
df_all['RecordID'] = df_all['RecordID'].astype('int')
df_all[df_all.columns[2:]] = df_all[df_all.columns[2:]].astype('float')

In [11]:
df_all.head()

,RecordID,TimeStamp,Albumin,ALP,ALT,AST,Bilirubin,BUN,Cholesterol,Creatinine,...,Platelets,RespRate,SaO2,SysABP,Temp,TroponinI,TroponinT,Urine,WBC,Weight
0,134389,00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,90.0
1,134389,00:10:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2,134389,00:11:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,134389,00:25:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,146.0,NaN,NaN,NaN,NaN,NaN,NaN
4,134389,00:26:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
feat_df = pd.DataFrame(columns=df_all.columns)
time_i = time.time()
i = 0
for id_ in unq_ids:
    if i%100 == 0:
        print i, time_i - time.time()
    i = i+1
    feat_id = df_all.loc[df_all['RecordID'] == int(id_)].resample('1H', on='TimeStamp').agg(np.nanmean)
    feat_id['RecordID'] = int(id_) 
    feat_df = feat_df.append(feat_id)

0 -0.000353097915649


/home/alok/.local/lib/python2.7/site-packages/numpy/core/_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)
/home/alok/.local/lib/python2.7/site-packages/numpy/core/_methods.py:70: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/alok/.local/lib/python2.7/site-packages/numpy/lib/nanfunctions.py:703: RuntimeWarning: Mean of empty slice
  warnings.warn("Mean of empty slice", RuntimeWarning)


100 -18.1895339489
200 -35.3569099903
300 -52.6505670547
400 -70.1401410103
500 -87.5940339565
600 -105.208595991
700 -122.957825899
800 -140.919806004
900 -158.953768015
1000 -177.088407993
1100 -195.390492916
1200 -213.767745972
1300 -232.589564085
1400 -251.274169922
1500 -270.169130087
1600 -288.991797924
1700 -307.873064041
1800 -326.787590981
1900 -345.828412056
2000 -364.992777109
2100 -384.259768963
2200 -403.61945796
2300 -423.460330963
2400 -444.138509989
2500 -465.087156057
2600 -486.32073307
2700 -507.943531036
2800 -529.51215601
2900 -551.918291092
3000 -577.562058926
3100 -601.045972109
3200 -628.100054026
3300 -653.328186989
3400 -677.446828127
3500 -703.480246067
3600 -730.158224106
3700 -753.363950014
3800 -777.042956114
3900 -800.549055099


In [13]:
feat_df['TimeStamp'] = feat_df.index

In [14]:
feat_df = feat_df.reset_index(drop=True)

In [15]:
feat_df.loc[feat_df.RecordID==134389.0]

,RecordID,TimeStamp,Albumin,ALP,ALT,AST,Bilirubin,BUN,Cholesterol,Creatinine,...,Platelets,RespRate,SaO2,SysABP,Temp,TroponinI,TroponinT,Urine,WBC,Weight
0,134389.0,0 days 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,88.333333,NaN,NaN,NaN,80.0,NaN,90.0
1,134389.0,0 days 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,121.000000,35.3,NaN,NaN,80.0,NaN,NaN
2,134389.0,0 days 02:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,117.000000,37.8,NaN,NaN,300.0,NaN,NaN
3,134389.0,0 days 03:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,142.000000,NaN,NaN,NaN,200.0,NaN,NaN
4,134389.0,0 days 04:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,146.000000,NaN,NaN,NaN,160.0,NaN,NaN
5,134389.0,0 days 05:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,135.000000,39.5,NaN,NaN,NaN,NaN,NaN
6,134389.0,0 days 06:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,118.000000,NaN,NaN,NaN,190.0,NaN,NaN
7,134389.0,0 days 07:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,129.000000,38.9,NaN,NaN,75.0,NaN,NaN
8,134389.0,0 days 08:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,101.000000,NaN,NaN,NaN,NaN,NaN,NaN
9,134389.0,0 days 09:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
feat_df.shape

(196000, 39)

In [17]:
with open('feat_df.pickle', 'wb') as handle:
    pickle.dump(feat_df, handle, protocol = pickle.HIGHEST_PROTOCOL)

In [18]:
feat_df_backfill = pd.DataFrame(columns=feat_df.columns)
time_i = time.time()
i = 0
for id_ in unq_ids:
    if i%100 == 0:
        print i, time_i - time.time()
    i = i+1
    feat_id_backfill = feat_df.loc[feat_df['RecordID'] == int(id_)].fillna(method='bfill')
    #feat_id_backfill = feat_df.loc[df_all['RecordID'] == int(id_)].fillna(method='ffill')
    feat_df_backfill = feat_df_backfill.append(feat_id_backfill)

0 -0.000507831573486
100 -0.322073936462
200 -0.82513999939
300 -1.50400686264
400 -2.30185079575
500 -3.24199199677
600 -4.31414580345
700 -5.50248098373
800 -6.90441679955
900 -8.49845790863
1000 -10.1756248474
1100 -11.9551358223
1200 -13.8204329014
1300 -15.7404909134
1400 -17.8110029697
1500 -20.0981738567
1600 -22.4555299282
1700 -24.8351619244
1800 -27.7576568127
1900 -30.3504137993
2000 -32.9520769119
2100 -35.6097187996
2200 -38.3509709835
2300 -41.5539839268
2400 -45.7372908592
2500 -50.1681330204
2600 -54.7888028622
2700 -59.5807008743
2800 -64.5097579956
2900 -69.6154270172
3000 -74.8973858356
3100 -80.3770449162
3200 -85.9981079102
3300 -91.760504961
3400 -97.6907889843
3500 -104.135854006
3600 -111.381739855
3700 -117.932705879
3800 -124.606580019
3900 -131.516923904


In [19]:
feat_df_ffill = pd.DataFrame(columns=feat_df.columns)
time_i = time.time()
i = 0
for id_ in unq_ids:
    if i%100 == 0:
        print i, time_i - time.time()
    i = i+1
    #feat_id_backfill = feat_df.loc[df_all['RecordID'] == int(id_)].fillna(method='bfill')
    feat_id_ffill = feat_df_backfill.loc[feat_df_backfill['RecordID'] == int(id_)].fillna(method='ffill')
    feat_df_ffill = feat_df_ffill.append(feat_id_ffill)

0 -0.00156688690186
100 -0.375442028046
200 -0.862320899963
300 -1.50161194801
400 -2.34853601456
500 -3.22832489014
600 -4.23792791367
700 -5.4275200367
800 -6.92588400841
900 -8.53722906113
1000 -10.1333549023
1100 -12.0877439976
1200 -14.1354558468
1300 -16.3699109554
1400 -18.4720740318
1500 -20.818641901
1600 -23.1764469147
1700 -25.5646760464
1800 -28.0228998661
1900 -30.5073988438
2000 -33.0842180252
2100 -35.7574169636
2200 -38.6778168678
2300 -41.9518759251
2400 -46.3508758545
2500 -50.8328359127
2600 -55.4488198757
2700 -60.2487578392
2800 -65.1884698868
2900 -70.3428678513
3000 -75.6542508602
3100 -81.1221029758
3200 -87.0555858612
3300 -92.8940038681
3400 -98.8895139694
3500 -105.022865057
3600 -111.292875051
3700 -117.826614857
3800 -124.477159977
3900 -131.281643867


In [35]:
normal_val = [-1]*39

In [36]:
i = 2
for col in feat_df_ffill.columns[2:]:
    #print i
    feat_df_ffill[col] = feat_df_ffill[col].fillna(normal_val[i])
    i = i+1

In [37]:
from sklearn.preprocessing import MinMaxScaler
X = MinMaxScaler().fit_transform(feat_df_ffill[feat_df_ffill.columns[2:]])

In [38]:
feat_df_ffill[feat_df_ffill.columns[2:]] = X

In [72]:
feat_df_ffill1 = feat_df_ffill.sort(['RecordID','TimeStamp'], ascending=[True,True])
feat_df_ffill1 = feat_df_ffill1.reset_index(drop=True)

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


In [57]:
outcomes = pd.read_csv("Outcomes-a.txt", usecols=[0,5])

In [74]:
outcomes = outcomes.sort(['RecordID'])

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


In [76]:
matrix3D = np.array(feat_df_ffill1.drop(['RecordID', 'TimeStamp'], 1))

In [77]:
matrix3D = np.array(matrix3D).reshape((4000, 49, 37))

In [79]:
with open('preprocess.pickle', 'wb') as handle:
    pickle.dump([matrix3D, outcomes], handle, protocol = pickle.HIGHEST_PROTOCOL)

In [81]:
outcomes.to_csv("preprocess/outcomes.csv", index=False)

In [86]:
feat_df_ffill1.to_csv("preprocess/feat_df_ffill1.csv", index=False)